In [6]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    ChatPromptTemplate,
)



In [7]:
# from langfuse.callback import CallbackHandler
# handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [8]:
GENERATIONS_PATH = "../datasets/baseline_zero_shot_gpt4_50_50_merged.csv"
MODEL = "gpt-3.5-turbo-0301"

In [9]:
data = pd.read_csv(GENERATIONS_PATH)

data["whole_output"] = ""


if "similarity" not in data.columns:
    data["similarity"] = ""
if "confidence" not in data.columns:
    data["confidence"] = ""
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output
0,0,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:50.732Z,2023-10-21T14:57:51.384Z,Are the following two products the same?\nProd...,False,False,True,FN,30%,50%,
1,1,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:50.131Z,2023-10-21T14:57:50.731Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,
2,2,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:49.548Z,2023-10-21T14:57:50.129Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,
3,3,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:49.042Z,2023-10-21T14:57:49.546Z,Are the following two products the same?\nProd...,False,False,True,FN,60%,80%,
4,4,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:48.283Z,2023-10-21T14:57:49.041Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:09.552Z,2023-10-21T14:55:10.190Z,Are the following two products the same?\nProd...,False,False,False,NONE,10%,40%,
217,217,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:08.802Z,2023-10-21T14:55:09.545Z,Are the following two products the same?\nProd...,False,False,False,NONE,50%,70%,
218,218,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:07.965Z,2023-10-21T14:55:08.800Z,Are the following two products the same?\nProd...,False,False,False,NONE,10%,50%,
219,219,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:07.028Z,2023-10-21T14:55:07.963Z,Are the following two products the same?\nProd...,False,False,False,NONE,25%,50%,


In [10]:


task = """ Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Similarity: 75%
Confidence: 60%
"""


STEP_CONFIDENCE_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", task),
        ])





In [11]:
# create an empty list
missed = []
def get_confidence_score(start, end):
    llm = ChatOpenAI(model_name=MODEL, temperature=0, max_tokens=100)
    chain = LLMChain(llm=llm, prompt=STEP_CONFIDENCE_PROMPT)
    for i in range(start, end):
        print(i)
        user_prompt = data["prompt"][i]
        ai_answer = data["completion"][i]
        try:
            output = str(chain.run(user_prompt=user_prompt, ai_answer=ai_answer))
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_CONFIDENCE_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        splitted_output = output.split("\n")
        try:
            similarity = splitted_output[0].split(": ")[1]
            confidence = splitted_output[1].split(": ")[1]
            data["similarity"][i] = similarity
            data["confidence"][i] = confidence
            data["whole_output"][i] = output
        except:
            data["similarity"][i] = output
            data["confidence"][i] = output
            missed.append(i)



In [12]:
get_confidence_score(0, len(data))

0
System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none

Answer with 'True' or 'False'.
AI: False
Human:  Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Similarity: 75%
Confidence: 60%

----
True
Similarity: 30%
Confidence: 50%
-----------
1


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD
Description: Refined and redesigned; ready to shred any lineRamped and siped tread to roll fast and track straight under brakingSingle-ply, DoubleDown, and DH casings available3C MaxxTerra, 3C MaxxGrip, and Super Tacky available on select modelsWide trail (WT) ca
Product 2: Title: Maxxis Minion DHR II 29\" DD/3C/TR
Description: Minion DHR II 29 x 2.35 tubeless ready tyre with Double down casing and 3C Maxx Terra.For our full range of Bike Accessories visit Evans Cycles

Answer with 'True' or 'False'.
AI: True
Human:  Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Similarity: 75%
Confidence: 60%

----
True
Similarity: 90%
Confidence: 80%
-------

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Traveler's Company 014. CRAFT PAPER (64PG) MIDORI TRAVELER'S NOTEBOOK
Description: none
Product 2: Title: Refill #014: Kraft Paper Notebook
Description: Cardboard Paper notebook refill for the Traveler's Notebook.Various cuttings, stickers, photos, scribbles and more sophisticated drawings suit it well.Contains 64 pages and its dimensions (210 x 110 mm) make it suitable for the classic version, not f

Answer with 'True' or 'False'.
AI: True
Human:  Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Similarity: 75%
Confidence: 60%

----
True
Similarity: 100%
Confidence: 90%
-----------
3


...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: Instax Mini Rainbow Film 10 Pack
Description: Brighten the place up with this pack of rainbow coloured film! Top tip: one of these would look fab in our Instax Mini Acrylic Photo Block!Includes pack of 10 shots
Product 2: Title: 1 Fujifilm Instax Square Film Black Frame
Description: none

Answer with 'True' or 'False'.
AI: False
Human:  Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Similarity: 75%
Confidence: 60%

----
False
Similarity: 25%
Confidence: 50%
-----------
220


/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

System: You are a helpful AI.
Human: Are the following two products the same?
Product 1: Title: DJI Mavic Mini Drone Fly More Combo
Description: Mavic Mini, the everyday flycam. Open your sky with the compact, yet powerful Mavic Mini. Weighing under 250 grams, together with the DJI Fly APP, you'll enjoy a simplified flying experience that is almost as light as the average smartphone.
Product 2: Title: Fujifilm Instax Mini Instant Film - Monochrome (10 Shots)
Description: Monochrome (Black & White) Instant Film for Fujifilm Mini Instant Cameras10 Shots per PackISO 800High Gloss FinishSuperb Grain QualityFast Image FormationGreater Resistance to Age-Induced ChangesSingle-plastic Composition for Easier Disposal

Answer with 'True' or 'False'.
AI: False
Human:  Provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which inclu

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_28485/3883228377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [13]:
print(missed)

[180, 213]


In [14]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output
0,0,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:50.732Z,2023-10-21T14:57:51.384Z,Are the following two products the same?\nProd...,False,False,True,FN,30%,50%,Similarity: 30%\nConfidence: 50%
1,1,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:50.131Z,2023-10-21T14:57:50.731Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,Similarity: 90%\nConfidence: 80%
2,2,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:49.548Z,2023-10-21T14:57:50.129Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,Similarity: 100%\nConfidence: 90%
3,3,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:49.042Z,2023-10-21T14:57:49.546Z,Are the following two products the same?\nProd...,False,False,True,FN,50%,70%,Similarity: 50%\nConfidence: 70%
4,4,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:57:48.283Z,2023-10-21T14:57:49.041Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,Similarity: 100%\nConfidence: 90%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:09.552Z,2023-10-21T14:55:10.190Z,Are the following two products the same?\nProd...,False,False,False,NONE,10%,40%,Similarity: 10%\nConfidence: 40%
217,217,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:08.802Z,2023-10-21T14:55:09.545Z,Are the following two products the same?\nProd...,False,False,False,NONE,50%,70%,Similarity: 50%\nConfidence: 70%
218,218,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:07.965Z,2023-10-21T14:55:08.800Z,Are the following two products the same?\nProd...,False,False,False,NONE,10%,50%,Similarity: 10%\nConfidence: 50%
219,219,7bd1339f-b49c-403d-a2d9-20672b315990,gpt-4-0613,2023-10-21T14:55:07.028Z,2023-10-21T14:55:07.963Z,Are the following two products the same?\nProd...,False,False,False,NONE,25%,50%,Similarity: 25%\nConfidence: 50%


In [15]:
data.to_csv(GENERATIONS_PATH, index=False)
